In [ ]:
# # saves you having to use print as all exposed variables are printed in the cell
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# suppress warning message
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.compute as pc
from tqdm.autonotebook import tqdm, trange

/var/folders/02/938l_81s1b3g_95s09yvbqj00000gq/T/ipykernel_73763/133596935.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


---------

In [4]:
from nn_rag import Knowledge
from nn_rag.components.commons import Commons

2024-07-15 12:25:05.040100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# create instance of capability
kn = Knowledge.from_memory()

In [10]:
from spacy.language import Language
import spacy
from tqdm.auto import tqdm

@Language.component("set_custom_boundaries")
def set_custom_boundaries(document):
    for token in tqdm(document[:-1], total=len(document)-1, desc='building paragraphs'):
        if token in ['.','?','!'] and document[token.i + 1].is_title:
            document[token.i + 1].is_sent_start = True
        return document

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("set_custom_boundaries", before="parser")
doc = nlp("This is a sentence. This is another sentence! Is this the last? This is another one.")
print("Result", [sent.text for sent in doc.sents])


building paragraphs:   0%|          | 0/19 [00:00<?, ?it/s]

Result ['This is a sentence.', 'This is another sentence!', 'Is this the last?', 'This is another one.']
